# 建立inverted index

In [20]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk.stem
import math
from math import log
import json


tweets_text = []    #tweets_text中的元素为原每个tweet的正文
label = []    #label中的元素为原每个tweet的id
with open('tweets.txt', encoding='utf-8') as file:
    for line in file:
        tweet = json.loads(line)
        tweets_text.append(tweet['text'])
        label.append(tweet['tweetId'])

# 分词等预处理
def filter(text):
    # tokenization
    #text1 = text.replace('-', ' ')
    filtered1 = word_tokenize(text)
    #print(filtered1)

    # 去标点
    punctuation = [',', '<', '>', '.', "'s", '`', '~', '!', '@', '#', '$', '%', '^',
                   '&', '*', '(', ')', '-', '_', '=', '+', '[', ']', '{', '}', '\\', '|',
                   ':', ';', "\'", '/', '?',"\"","“","''","``"]
    filtered2 = [i for i in filtered1 if i not in punctuation]
    #print(filtered2)

    # Normalization

    # Stemming
    filtered3 = []
    s = nltk.stem.SnowballStemmer('english')
    for i in filtered2:
        # 大写都变为小写
        i.lower()
        filtered3.append(s.stem(i))
    #print(filtered3)

    # 去stopwords
    filtered4 = [w for w in filtered3 if w not in stopwords.words('english')]
    #print(filtered4)
    return(filtered4)


textword = []
for text in tweets_text:
    textword.append(filter(text))



# 计算文档平均长度
M = len(textword)   #总文档数
sum_length = 0
for text in textword:
    sum_length = sum_length + len(text)  # len(textword[i])就是某一文档的长度
avdl = sum_length/M
print('avdl = ', avdl)

#建立inverted index
dict1 = {}
label_num = 0
for word_list in textword:
    for word in word_list:
        if word not in dict1:
            dict1[word] = []
            dict1[word].append([label_num, word_list.count(word)])
        elif label != dict1[word][-1][0]:
            dict1[word].append([label_num, word_list.count(word)])
    label_num = label_num + 1

avdl =  12.324374754484746


# 建立queryid到query内容的字典

In [9]:
num_query = {}
with open('querynumtoquery.txt', encoding='utf-8') as file:
    for line in file:
        num_query[int(line[:3])] = line[4:-1]
print(num_query)

{171: 'Ron Weasley birthday', 172: 'Merging of US Air and American', 173: 'muscle pain from statins', 174: 'Hubble oldest star', 175: 'commentary on naming storm Nemo', 176: 'book club members', 177: 'Boko Haram kidnapped French tourists', 178: 'Tiger Woods regains title', 179: 'care of Iditarod dogs', 180: 'Sherlock Elementary BBC CBS', 181: 'Costa Concordia shipwreck', 182: 'Chinua Achebe death', 183: 'Evernote hacked', 184: 'Election of Hugo Chavez successor', 185: 'National Zoo Panda insemination', 186: 'Dorner truck compensation', 187: 'Pope washed Muslims feet', 188: 'Bombing police headquarters Kirkuk', 189: 'injuries by pets', 190: 'Organized crime sports doping Australia', 191: 'Irish laundries apology', 192: 'whooping cough epidemic', 193: 'Bulgarian protesters self immolate', 194: 'cherry blossom Washington', 195: 'Argo wins Oscar', 196: 'US fines Google over Street View ', 197: 'Mad Men season 6', 198: 'Hostess bought by Apollo', 199: 'Ed Koch death', 200: 'UK passes marria

# 检索、打分并排序输出

In [63]:
b = 0.6
k = 1
#检索、打分
for num in range(171,226):
    query = num_query[num]
    query_word = filter(query)
#     print('query预处理: ', query_word)
    score_list = []
    for word in query_word:
        if word in dict1.keys():
            for d in dict1[word]:
                # PLNVSM
#                 sim = query_word.count(word) * log(1 + log(1+d[1], math.e), math.e) / \
#                         (1 - b + b * len(textword[d[0]])/avdl) * log((M+1)/len(dict1[word]), 2)
                # BM25
                sim = query_word.count(word) * (k+1)*d[1]/(d[1]+k*(1-b+b*len(textword[d[0]])/avdl)) \
                        * log((M+1)/len(dict1[word]), 2)
                judge = True
                for i in score_list:
                    if i[0] == d[0]:
                        judge = False
                        i[1] = i[1] + sim
                if judge:
                    score_list.append([d[0], sim])     # 【文档编号，文档评分】
    # 排序输出
    score_list.sort(key=lambda x:x[1], reverse = True)
    # 最多只输出评分前100的tweetId
    score_list100 = score_list[0:min(100,len(score_list))]
    s = []   
    c = []
    for i in score_list:
        s.append(i[1])
        c.append(label[i[0]])
#     print('文档评分: ', s)
#     print('对应tweetId: ', c)
    # write
    with open('result.txt','a+') as f:
        for i in range(len(c)):
            f.write(str(num)+' '+c[i]+'\n')